<a href="https://colab.research.google.com/github/chuuxter/Algoritmos-de-Clasificion-de-estilos-VAK-Balanceo-Progresivo-/blob/main/MODELOS_DE_CLASIFICACION_(BALANCEO_GRADUAL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTAR LIBRERIAS NECESARIAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from google.colab import files

print(" Librerías cargadas correctamente")



 Librerías cargadas correctamente


In [ ]:

# CARGA DE ARCHIVOS
# Subir el archivo desde tu computadora

uploaded = files.upload()

# Leer el archivo Excel
df = pd.read_excel('Dataset Modelo de Clasificación VARK (ESTILOS SIMP.).xlsx')

print("Dimensiones de tu dataset:")
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
print("\nPrimeras 10 filas:")
print(df.head(10))

print("\nColumnas disponibles:")
print(df.columns.tolist())


Saving Dataset Modelo de Clasificación VARK (ESTILOS SIMP.).xlsx to Dataset Modelo de Clasificación VARK (ESTILOS SIMP.) (1).xlsx
Dimensiones de tu dataset:
Filas: 257, Columnas: 72

Primeras 10 filas:
  GRADO SECCION  ORDEN  V1  A1  R1  K1  V2  A2  R2  ...  K15  V16  A16  R16  \
0   4TO       A     10   0   1   0   1   0   0   0  ...    1    0    1    1   
1   4TO       A      7   0   0   1   0   0   0   0  ...    1    0    0    0   
2   4TO       A     12   0   0   0   0   1   0   1  ...    0    0    0    0   
3   4TO       A     12   0   1   0   0   0   0   0  ...    0    0    1    0   
4   4TO       A     12   0   0   0   1   0   0   0  ...    0    0    0    0   
5   4TO       A     12   0   1   0   1   0   0   1  ...    0    0    0    1   
6   4TO       A     13   0   0   0   1   1   0   0  ...    0    0    0    0   
7   4TO       A      5   0   0   1   0   0   0   0  ...    1    0    0    0   
8   4TO       A      3   1   0   0   0   1   1   0  ...    0    1    0    0   
9   4T

In [ ]:
# PREPARACION DEL DATASET PARA ALIMENTAR A LOS ALGORITMOS

# Eliminar columnas innecesarias
columnas_a_eliminar = ['GRADO', 'SECCION', 'ORDEN', 'COUNT V', 'COUNT A', 'COUNT R', 'COUNT K']
df_ml = df.drop(columns=columnas_a_eliminar, errors='ignore')

print("Columnas eliminadas: GRADO, SECCION, ORDEN, COUNT V, COUNT A, COUNT R, COUNT K")
print(f"Dataset para ML: {df_ml.shape[0]} filas, {df_ml.shape[1]} columnas")

# Mostrar fragmento con valores binarios (0 y 1)
print("\n" + "="*60)
print(" FRAGMENTO DEL DATASET (valores 0 y 1)")
print("="*60)
columnas_muestra = ['V1', 'A1', 'R1', 'K1', 'V2', 'A2', 'R2', 'K2', 'V16', 'A16', 'R16', 'K16', 'ESTILO']
muestra_df = df_ml[columnas_muestra].head(8)
print(muestra_df.to_string())

df_ml = df_ml.dropna(subset=['ESTILO'])

print("Filas restantes después de eliminar ESTILO vacío:")
print(df_ml.shape[0])

print("\nNueva distribución de estilos:")
print(df_ml['ESTILO'].value_counts())


Columnas eliminadas: GRADO, SECCION, ORDEN, COUNT V, COUNT A, COUNT R, COUNT K
Dataset para ML: 257 filas, 65 columnas

 FRAGMENTO DEL DATASET (valores 0 y 1)
   V1  A1  R1  K1  V2  A2  R2  K2  V16  A16  R16  K16 ESTILO
0   0   1   0   1   0   0   0   1    0    1    1    1      K
1   0   0   1   0   0   0   0   1    0    0    0    1      K
2   0   0   0   0   1   0   1   0    0    0    0    0      V
3   0   1   0   0   0   0   0   0    0    1    0    0      A
4   0   0   0   1   0   0   0   1    0    0    0    1      K
5   0   1   0   1   0   0   1   0    0    0    1    0      K
6   0   0   0   1   1   0   0   0    0    0    0    1      A
7   0   0   1   0   0   0   0   1    0    0    0    1      K
Filas restantes después de eliminar ESTILO vacío:
236

Nueva distribución de estilos:
ESTILO
K    173
A     40
R     16
V      7
Name: count, dtype: int64


In [ ]:
# Función para alfa de Cronbach
def cronbach_alpha(df_items):
    df_items = df_items.dropna()  # eliminar filas con NaN
    k = df_items.shape[1]         # número de ítems
    var_sum = df_items.var(axis=0, ddof=1).sum()  # suma de varianzas individuales
    var_total = df_items.sum(axis=1).var(ddof=1)  # varianza del puntaje total
    alpha = (k / (k - 1)) * (1 - (var_sum / var_total))
    return alpha

# Definir columnas por estilo
estilos = {
    "Visual": [col for col in df_ml.columns if col.startswith('V')],
    "Auditivo": [col for col in df_ml.columns if col.startswith('A')],
    "Lectura/Escritura": [col for col in df_ml.columns if col.startswith('R')],
    "Kinestésico": [col for col in df_ml.columns if col.startswith('K')],
}

# Calcular alfa por estilo
for estilo, columnas in estilos.items():
    alpha = cronbach_alpha(df_ml[columnas])
    print(f" Alfa de Cronbach para {estilo}: {alpha:.4f}")

# Calcular alfa general (todo el cuestionario)
todas_columnas = estilos["Visual"] + estilos["Auditivo"] + estilos["Lectura/Escritura"] + estilos["Kinestésico"]
alpha_general = cronbach_alpha(df_ml[todas_columnas])
print(f"\n Alfa de Cronbach general del cuestionario: {alpha_general:.4f}")


 Alfa de Cronbach para Visual: 0.6188
 Alfa de Cronbach para Auditivo: 0.7113
 Alfa de Cronbach para Lectura/Escritura: 0.5790
 Alfa de Cronbach para Kinestésico: 0.7457

 Alfa de Cronbach general del cuestionario: 0.7428


In [ ]:
# DEFINICIÓN DE X E Y
X = df_ml.drop(columns=['ESTILO'])
y = df_ml['ESTILO']


In [ ]:
# DATASET ORIGINAL


datasets_para_modelos = {}
datasets_para_modelos["Original"] = (X, y)

print("Dataset ORIGINAL guardado.\n")


# BALANCEOS PERSONALIZADOS


cont = y.value_counts()
print("=== DISTRIBUCIÓN ORIGINAL ===")
print(cont, "\n")

resultados_balanceo = {}

ajustes = {
    "Compensado Suave":  {"K": -0.15, "A": +0.10, "R": +0.20, "V": +0.60},
    "Compensado Medio":  {"K": -0.30, "A": +0.20, "R": +0.40, "V": +1.0},
    "Compensado Fuerte": {"K": -0.45, "A": +0.30, "R": +0.60, "V": +2.40}
}

for nivel, pref in ajustes.items():

    print(f"\n {nivel}")

    nuevos_tamaños = {}
    for clase, cantidad in cont.items():
        pct = pref.get(clase, 0)
        nuevos_tamaños[clase] = max(1, int(cantidad * (1 + pct)))

    print("Objetivos:", nuevos_tamaños)

    # UNDERSAMPLING
    under_strategy = {c: min(nuevos_tamaños[c], cont[c]) for c in cont.index}
    rus = RandomUnderSampler(sampling_strategy=under_strategy, random_state=42)
    X_under, y_under = rus.fit_resample(X, y)

    # OVERSAMPLING
    over_strategy = {c: max(nuevos_tamaños[c], under_strategy[c]) for c in cont.index}
    ros = RandomOverSampler(sampling_strategy=over_strategy, random_state=42)
    X_bal, y_bal = ros.fit_resample(X_under, y_under)

    datasets_para_modelos[nivel] = (X_bal, y_bal)

    print("\nResultado final:")
    print(y_bal.value_counts())


Dataset ORIGINAL guardado.

=== DISTRIBUCIÓN ORIGINAL ===
ESTILO
K    173
A     40
R     16
V      7
Name: count, dtype: int64 


 Compensado Suave
Objetivos: {'K': 147, 'A': 44, 'R': 19, 'V': 11}

Resultado final:
ESTILO
K    147
A     44
R     19
V     11
Name: count, dtype: int64

 Compensado Medio
Objetivos: {'K': 121, 'A': 48, 'R': 22, 'V': 14}

Resultado final:
ESTILO
K    121
A     48
R     22
V     14
Name: count, dtype: int64

 Compensado Fuerte
Objetivos: {'K': 95, 'A': 52, 'R': 25, 'V': 23}

Resultado final:
ESTILO
K    95
A    52
R    25
V    23
Name: count, dtype: int64


In [ ]:
# Diccionario donde guardaremos las divisiones listas para modelos
splits = {}

for nombre, (X_base, y_base) in datasets_para_modelos.items():

    print(f"\n=== Procesando: {nombre} ===")

    X_train, X_test, y_train, y_test = train_test_split(
        X_base, y_base,
        test_size=0.2,
        random_state=42,
        stratify=y_base
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    splits[nombre] = {
        "X_train": X_train,
        "X_test": X_test,
        "y_train": y_train,
        "y_test": y_test,
        "X_train_scaled": X_train_scaled,
        "X_test_scaled": X_test_scaled
    }

    print("División y escalado completados.")



=== Procesando: Original ===
División y escalado completados.

=== Procesando: Compensado Suave ===
División y escalado completados.

=== Procesando: Compensado Medio ===
División y escalado completados.

=== Procesando: Compensado Fuerte ===
División y escalado completados.


In [ ]:
print("\n=== APLICANDO LABEL ENCODER Y TRATAMIENTO DE NULOS A TODOS LOS DATASETS ===")
for nombre in splits.keys():

    # TRATAMIENTO DE NULOS
    splits[nombre]["X_train"] = splits[nombre]["X_train"].fillna(0)
    splits[nombre]["X_test"]  = splits[nombre]["X_test"].fillna(0)
    splits[nombre]["X_train_scaled"] = np.nan_to_num(splits[nombre]["X_train_scaled"])
    splits[nombre]["X_test_scaled"]  = np.nan_to_num(splits[nombre]["X_test_scaled"])

    # LABEL ENCODER
    le = LabelEncoder()
    y_train_enc = le.fit_transform(splits[nombre]["y_train"])
    y_test_enc  = le.transform(splits[nombre]["y_test"])

    # Guardar dentro del mismo diccionario
    splits[nombre]["y_train_enc"] = y_train_enc
    splits[nombre]["y_test_enc"]  = y_test_enc
    splits[nombre]["label_encoder"] = le

    print(f"✔ Label encoding y tratamiento de nulos aplicado a: {nombre}")



=== APLICANDO LABEL ENCODER Y TRATAMIENTO DE NULOS A TODOS LOS DATASETS ===
✔ Label encoding y tratamiento de nulos aplicado a: Original
✔ Label encoding y tratamiento de nulos aplicado a: Compensado Suave
✔ Label encoding y tratamiento de nulos aplicado a: Compensado Medio
✔ Label encoding y tratamiento de nulos aplicado a: Compensado Fuerte


In [ ]:
#  TRATAMIENTO DE NULOS
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = np.nan_to_num(X_test_scaled)

In [ ]:

# ENTRENAMIENTO Y EVALUACIÓN


modelos = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(kernel='rbf', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "XGBoost": XGBClassifier(random_state=42, eval_metric='mlogloss')
}

resultados_globales = {}

for nombre_balanceo, (X_bal, y_bal) in datasets_para_modelos.items():
    print(f"\n\n==============================")
    print(f"BALANCEO ACTUAL: {nombre_balanceo}")
    print("==============================")

    # Mostrar distribución de clases
    print("Distribución de clases:")
    print(y_bal.value_counts())
    print("-----------------------------------")

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_bal, y_bal, test_size=0.2, random_state=42, stratify=y_bal
    )

    # Tratamiento de nulos
    X_train = X_train.fillna(0)
    X_test  = X_test.fillna(0)

    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    X_train_scaled = np.nan_to_num(X_train_scaled)
    X_test_scaled  = np.nan_to_num(X_test_scaled)

    # Label Encoding
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc  = le.transform(y_test)

    resultados_modelos = {}

    for nombre_modelo, modelo in modelos.items():
        print(f"  - Entrenando {nombre_modelo} ...")

        if nombre_modelo in ["SVM", "KNN"]:
            modelo.fit(X_train_scaled, y_train_enc)
            y_pred = modelo.predict(X_test_scaled)
        else:
            modelo.fit(X_train, y_train_enc)
            y_pred = modelo.predict(X_test)

        acc = accuracy_score(y_test_enc, y_pred)
        prec = precision_score(y_test_enc, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test_enc, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test_enc, y_pred, average='weighted', zero_division=0)

        resultados_modelos[nombre_modelo] = [acc, prec, rec, f1]

    resultados_globales[nombre_balanceo] = resultados_modelos

# ============================
# MOSTRAR RESULTADOS FINALES
# ============================

for balanceo, resultados in resultados_globales.items():
    df_result = pd.DataFrame(resultados, index=['Accuracy', 'Precision', 'Recall', 'F1']).T.round(4)
    print(f"\nRESULTADOS PARA: {balanceo}")
    print(df_result)

    # Mejor modelo según F1
    mejor_modelo = df_result['F1'].idxmax()
    mejor_f1 = df_result['F1'].max()
    print(f" MEJOR MODELO: {mejor_modelo} con F1 = {mejor_f1:.4f}")
    print("-----------------------------------")




BALANCEO ACTUAL: Original
Distribución de clases:
ESTILO
K    173
A     40
R     16
V      7
Name: count, dtype: int64
-----------------------------------
  - Entrenando Random Forest ...
  - Entrenando SVM ...
  - Entrenando KNN ...
  - Entrenando XGBoost ...


BALANCEO ACTUAL: Compensado Suave
Distribución de clases:
ESTILO
K    147
A     44
R     19
V     11
Name: count, dtype: int64
-----------------------------------
  - Entrenando Random Forest ...
  - Entrenando SVM ...
  - Entrenando KNN ...
  - Entrenando XGBoost ...


BALANCEO ACTUAL: Compensado Medio
Distribución de clases:
ESTILO
K    121
A     48
R     22
V     14
Name: count, dtype: int64
-----------------------------------
  - Entrenando Random Forest ...
  - Entrenando SVM ...
  - Entrenando KNN ...
  - Entrenando XGBoost ...


BALANCEO ACTUAL: Compensado Fuerte
Distribución de clases:
ESTILO
K    95
A    52
R    25
V    23
Name: count, dtype: int64
-----------------------------------
  - Entrenando Random Forest ...


In [ ]:

print("  MEJOR MODELO POR BALANCEO ")


for balanceo, resultados in resultados_globales.items():
    df_result = pd.DataFrame(resultados, index=['Accuracy', 'Precision', 'Recall', 'F1']).T
    mejor_modelo = df_result['F1'].idxmax()
    mejor_f1 = df_result['F1'].max()

    print(f" {balanceo}:")
    print(f"  → Mejor modelo: {mejor_modelo}")
    print(f"  → F1 score: {mejor_f1:.4f}\n")


  MEJOR MODELO POR BALANCEO 
 Original:
  → Mejor modelo: XGBoost
  → F1 score: 0.8053

 Compensado Suave:
  → Mejor modelo: XGBoost
  → F1 score: 0.9051

 Compensado Medio:
  → Mejor modelo: XGBoost
  → F1 score: 0.9007

 Compensado Fuerte:
  → Mejor modelo: XGBoost
  → F1 score: 0.9746



In [ ]:

#   REPORTE DETALLADO – XGBoost (Compensado Suave)


nivel = "Compensado Suave"

print(f"\n REPORTE DETALLADO – XGBoost ({nivel})")
print("=" * 70)

# Extraer los splits
X_train = splits[nivel]["X_train"]
X_test  = splits[nivel]["X_test"]
y_train = splits[nivel]["y_train"]
y_test  = splits[nivel]["y_test"]


#   1) LABEL ENCODER EXCLUSIVO PARA ESTE DATASET

le_xgb = LabelEncoder()
y_train_enc = le_xgb.fit_transform(y_train)
y_test_enc  = le_xgb.transform(y_test)


#   2) ENTRENAR XGBOOST (NO REQUIERE ESCALADO)

modelo_xgb = XGBClassifier(random_state=42, eval_metric='mlogloss')
modelo_xgb.fit(X_train, y_train_enc)

# Predicción
y_pred_enc = modelo_xgb.predict(X_test)


#   3) MÉTRICAS

acc = accuracy_score(y_test_enc, y_pred_enc)
prec = precision_score(y_test_enc, y_pred_enc, average='weighted', zero_division=0)
rec = recall_score(y_test_enc, y_pred_enc, average='weighted', zero_division=0)
f1 = f1_score(y_test_enc, y_pred_enc, average='weighted', zero_division=0)

print(f" Accuracy:   {acc:.2%}")
print(f" Precision:  {prec:.2%}")
print(f" Recall:     {rec:.2%}")
print(f" F1-Score:   {f1:.2%}")

#   4) REPORTE POR ESTILO

print("\n MÉTRICAS POR ESTILO DE APRENDIZAJE:")
labels = le_xgb.classes_
print(classification_report(y_test_enc, y_pred_enc, target_names=labels, digits=4))

print("\n Evaluación completada para XGBoost (balanceo suave).")



 REPORTE DETALLADO – XGBoost (Compensado Suave)
 Accuracy:   91.11%
 Precision:  91.53%
 Recall:     91.11%
 F1-Score:   90.51%

 MÉTRICAS POR ESTILO DE APRENDIZAJE:
              precision    recall  f1-score   support

           A     0.8889    0.8889    0.8889         9
           K     0.9062    0.9667    0.9355        30
           R     1.0000    0.5000    0.6667         4
           V     1.0000    1.0000    1.0000         2

    accuracy                         0.9111        45
   macro avg     0.9488    0.8389    0.8728        45
weighted avg     0.9153    0.9111    0.9051        45


 Evaluación completada para XGBoost (balanceo suave).
